You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of artificial intelligence (AI) and its impact on various industries
- Mention of the rapid advancements and innovations in AI technology

Key Points:
1. Latest Trends in AI:
- Machine learning and deep learning advancements
- Natural language processing and sentiment analysis
- AI-powered automation and robotics
- Ethical considerations in AI development

2. Key Players in AI:
- Google DeepMin

I now can give a great answer

Final Answer:
# The Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has transformed various industries with its innovative capabilities, driving significant advancements that shape the future. The continuous progress in AI technology has paved the way for groundbreaking innovations that enhance automation processes and deliver personalized experiences. AI stands at the forefront of driving change across sectors, revolutionizing the way businesses operate and interact with customers.

## Latest Trends in AI
Machine learning and deep learning have emerged as pivotal trends in AI, enabling systems to learn from data and continuously improve their performance. Natural language processing and sentiment analysis have also gained momentum, allowing machines to understand and interpret human language with remarkable accuracy. AI-powered automation and robotics are revolutionizing industries by optimizing oper

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has transformed various industries with its innovative capabilities, driving significant advancements that shape the future. The continuous progress in AI technology has paved the way for groundbreaking innovations that enhance automation processes and deliver personalized experiences. AI stands at the forefront of driving change across sectors, revolutionizing the way businesses operate and interact with customers.

## Latest Trends in AI
Machine learning and deep learning have emerged as pivotal trends in AI, enabling systems to learn from data and continuously improve their performance. Natural language processing and sentiment analysis have also gained momentum, allowing machines to understand and interpret human language with remarkable accuracy. AI-powered automation and robotics are revolutionizing industries by optimizing operations and enhancing efficiency. However, ethical considerations in AI development remain crucial, ensuring the responsible and unbiased utilization of AI technologies for the benefit of society.

## Key Players in AI
The AI landscape is dominated by key players such as Google DeepMind, renowned for its groundbreaking AI research and applications. IBM Watson leads in cognitive computing, providing AI solutions tailored for businesses across diverse sectors. Amazon Web Services and Microsoft Azure offer cloud-based AI services, empowering organizations to harness AI capabilities on a large scale. OpenAI, an AI research organization, focuses on developing safe and beneficial AI solutions for the global community.

## Noteworthy News in AI
Recent collaborations and partnerships in the AI industry have fueled innovation and accelerated the adoption of AI technologies. Breakthroughs in AI research and development continue to push the boundaries of what AI can achieve, opening up new possibilities for growth and optimization in industries such as healthcare, finance, and cybersecurity. AI applications are reshaping these sectors, offering unprecedented opportunities for advancement and efficiency.

## Conclusion
Staying informed about the latest trends and key players in the AI industry is crucial for professionals, technology enthusiasts, and students interested in AI. By keeping abreast of the latest news and advancements in AI technology, individuals can navigate the dynamic landscape of artificial intelligence and make informed decisions. Subscribing to industry newsletters and participating in AI conferences are valuable ways to stay engaged and updated in the ever-evolving world of AI. Share your thoughts and insights on the latest AI developments in the comments section below.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Employment rate"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Employment rate.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan

Title: Understanding Employment Rate Trends: A Comprehensive Analysis

Outline:
I. Introduction
- Definition of employment rate
- Importance of understanding employment rate trends
- Preview of key points to be discussed

II. Latest Trends in Employment Rate
- Overview of the current global employment rate
- Comparison of employment rates across different industries
- Impact of technological advancements on employment trends

III. Key Players in the Employment Market
- Analysis of top companies creatin

I now can give a great answer

Final Answer: 

# Understanding Employment Rate Trends: A Comprehensive Analysis

## Introduction

The employment rate serves as a vital indicator of a country's economic well-being, measuring the percentage of the workforce that is currently employed. It is crucial for policymakers, businesses, and individuals to grasp employment rate trends in order to effectively navigate the job market. In this blog post, we will delve into the latest trends in employment rates, analyze key players in the employment market, highlight noteworthy news, and provide valuable insights for our target audience.

## Latest Trends in Employment Rate

At present, the global employment rate stands at [insert current percentage], reflecting the impact of recent economic changes and technological advancements. Various industries exhibit different employment rates, with sectors such as technology witnessing growth while more traditional industries encounter challenges. The emergenc

In [14]:
Markdown(result)

# Understanding Employment Rate Trends: A Comprehensive Analysis

## Introduction

The employment rate serves as a vital indicator of a country's economic well-being, measuring the percentage of the workforce that is currently employed. It is crucial for policymakers, businesses, and individuals to grasp employment rate trends in order to effectively navigate the job market. In this blog post, we will delve into the latest trends in employment rates, analyze key players in the employment market, highlight noteworthy news, and provide valuable insights for our target audience.

## Latest Trends in Employment Rate

At present, the global employment rate stands at [insert current percentage], reflecting the impact of recent economic changes and technological advancements. Various industries exhibit different employment rates, with sectors such as technology witnessing growth while more traditional industries encounter challenges. The emergence of automation and artificial intelligence has further influenced employment trends, introducing both opportunities and disruptions to the job market.

## Key Players in the Employment Market

Prominent companies play a pivotal role in creating job opportunities and shaping the employment landscape. Government policies and initiatives also exert influence on employment rates, as regulations impact hiring practices and labor markets. By examining successful employment initiatives and case studies, valuable insights can be gained into effective strategies for fostering job growth and workforce development.

## Noteworthy News on Employment Rate

Recent studies on employment rate projections provide valuable insights into future job market trends and obstacles. Job seekers confront challenges such as skill mismatches and evolving job requirements, underscoring the importance of continuous learning and adaptability. Success stories of individuals surmounting employment hurdles serve as sources of inspiration, showcasing the resilience of the human spirit in the face of adversity.

## Target Audience Analysis

Our target audience comprises working professionals, job seekers, and students with a keen interest in career advancement and job market trends. Understanding the economic ramifications of employment rate trends is vital for addressing issues like unemployment, job insecurity, and skill development requirements. By furnishing actionable information and resources, our aim is to empower our audience to make well-informed decisions about their careers and successfully navigate the job market.

## Conclusion

In conclusion, a comprehensive understanding of employment rate trends is indispensable for maneuvering through the intricate and ever-evolving job market. By scrutinizing the latest trends, key players, and noteworthy news on employment rates, valuable insights can be gleaned into the factors shaping the workforce of tomorrow. Through continual learning, adaptability, and collaboration, challenges can be overcome and opportunities seized for career growth and achievement.

Remember to stay abreast of employment rate trends, utilize resources for skill enhancement and job search support, and share your employment experiences and insights with our community. Together, we can forge a brighter future for all individuals striving to excel in the dynamic realm of work.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).